## Model Building and Training for 1st Proactive Strategy

In [1]:
# Importing necessary libraries for model building and training
import os
import gc
import utils
import numpy as np
import pandas as pd
import csv
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import callbacks
from tabulate import tabulate

#### Loading the sequences

In [2]:
# Defining the paths
base_path = "data/processed"
datasets = ["rb", "gl", "tc"]
splits = ["train", "valid"]

In [3]:
# Load the sequence arrays for model training
loaded_data = {data: [utils.load_sequences(base_path, data, split) for split in splits] for data in datasets}
(trd_rb, trl_rb), (vad_rb, val_rb) = loaded_data["rb"]
(trd_gl, trl_gl), (vad_gl, val_gl) = loaded_data["gl"]
(trd_tc, trl_tc), (vad_tc, val_tc) = loaded_data["tc"]

Loaded train data for rb: (9230, 30, 71), (9230,)
Loaded valid data for rb: (1707, 30, 71), (1707,)
Loaded train data for gl: (9188, 30, 71), (9188,)
Loaded valid data for gl: (1707, 30, 71), (1707,)
Loaded train data for tc: (4742, 30, 71), (4742,)
Loaded valid data for tc: (1707, 30, 71), (1707,)


#### Building the models

In [4]:
# Function to build the LSTM model for fault detection
def build_advanced_lstm_model(input_shape, task_type="classification"):
    if task_type not in ["classification", "regression"]:
        raise ValueError("task_type must be either 'classification' or 'regression'.")

    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, kernel_size=3, activation="relu", padding="same", kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.001), dropout=0.3))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.001), dropout=0.3))(x)
    x = layers.BatchNormalization()(x)
    
    attention = layers.Attention()([x, x])
    x = layers.Concatenate()([x, attention])
    
    x = layers.Bidirectional(layers.LSTM(64, kernel_regularizer=regularizers.l2(0.001), dropout=0.4))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    
    if task_type == "classification":
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = models.Model(inputs, outputs)
        model.compile(
            optimizer=tf.keras.optimizers.AdamW(),
            loss="binary_crossentropy",
            metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
        )
    elif task_type == "regression":
        outputs = layers.Dense(1, activation="linear")(x)
        model = models.Model(inputs, outputs)
        model.compile(
            optimizer=tf.keras.optimizers.AdamW(),
            loss="log_cosh",
            metrics=["mae", tf.keras.metrics.RootMeanSquaredError(name="rmse")]
        )
    
    model.summary()
    return model

In [5]:
# Setting the input shapes for the models
input_shape_rb = trd_rb.shape[1:]
input_shape_gl = trd_gl.shape[1:]
input_shape_tc = trd_tc.shape[1:]

# Checking the input shapes of the datasets
print(f'Input shape for the model "Robot Protective Stop" {input_shape_rb}')
print(f'Input shape for the model "Grip Lost" {input_shape_gl}')
print(f'Input shape for the model "Tool Current" {input_shape_tc}')

Input shape for the model "Robot Protective Stop" (30, 71)
Input shape for the model "Grip Lost" (30, 71)
Input shape for the model "Tool Current" (30, 71)


In [6]:
# Building the 1st model "Robot Protective Stop" [Classification]
model_rb = build_advanced_lstm_model(input_shape_rb)
utils.save_model_summary(model_rb, "models/rb/model_summary_rb.txt")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 71)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 30, 64)            │          13,696 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 30, 64)            │             256 │ conv1d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d (MaxPooling1D)  │ (None, 15, 64)            │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 15, 512)           │         657,408 │ max_pooling1d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 15, 512)           │           2,048 │ bidirectional[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_1               │ (None, 15, 256)           │         656,384 │ batch_normalization_1[0][… │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 15, 256)           │           1,024 │ bidirectional_1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention (Attention)         │ (None, 15, 256)           │               0 │ batch_normalization_2[0][… │
│                               │                           │                 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 15, 512)           │               0 │ batch_normalization_2[0][… │
│                               │                           │                 │ attention[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_2               │ (None, 128)               │         295,424 │ concatenate[0][0]          │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 128)               │             512 │ bidirectional_2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 256)               │          33,024 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 1,660,033 (6.33 MB)

 Trainable params: 1,658,113 (6.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Model summary saved at models/rb/model_summary_rb.txt


In [7]:
# Buidling the 2nd model "Grip Lost" [Classification]
model_gl = build_advanced_lstm_model(input_shape_gl)
utils.save_model_summary(model_gl, "models/gl/model_summary_gl.txt")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 30, 71)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 30, 64)            │          13,696 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 30, 64)            │             256 │ conv1d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_1               │ (None, 15, 64)            │               0 │ batch_normalization_4[0][… │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_3               │ (None, 15, 512)           │         657,408 │ max_pooling1d_1[0][0]      │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_5         │ (None, 15, 512)           │           2,048 │ bidirectional_3[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_4               │ (None, 15, 256)           │         656,384 │ batch_normalization_5[0][… │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_6         │ (None, 15, 256)           │           1,024 │ bidirectional_4[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_1 (Attention)       │ (None, 15, 256)           │               0 │ batch_normalization_6[0][… │
│                               │                           │                 │ batch_normalization_6[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 15, 512)           │               0 │ batch_normalization_6[0][… │
│                               │                           │                 │ attention_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_5               │ (None, 128)               │         295,424 │ concatenate_1[0][0]        │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_7         │ (None, 128)               │             512 │ bidirectional_5[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 1,660,033 (6.33 MB)

 Trainable params: 1,658,113 (6.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Model summary saved at models/gl/model_summary_gl.txt


In [8]:
# Building the 3rd model "Tool Current" [Regression]
model_tc = build_advanced_lstm_model(input_shape_tc, "regression")
utils.save_model_summary(model_tc, "models/tc/model_summary_tc.txt")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 30, 71)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 30, 64)            │          13,696 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_8         │ (None, 30, 64)            │             256 │ conv1d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_2               │ (None, 15, 64)            │               0 │ batch_normalization_8[0][… │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_6               │ (None, 15, 512)           │         657,408 │ max_pooling1d_2[0][0]      │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_9         │ (None, 15, 512)           │           2,048 │ bidirectional_6[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_7               │ (None, 15, 256)           │         656,384 │ batch_normalization_9[0][… │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_10        │ (None, 15, 256)           │           1,024 │ bidirectional_7[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_2 (Attention)       │ (None, 15, 256)           │               0 │ batch_normalization_10[0]… │
│                               │                           │                 │ batch_normalization_10[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_2 (Concatenate)   │ (None, 15, 512)           │               0 │ batch_normalization_10[0]… │
│                               │                           │                 │ attention_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_8               │ (None, 128)               │         295,424 │ concatenate_2[0][0]        │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_11        │ (None, 128)               │             512 │ bidirectional_8[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 1,660,033 (6.33 MB)

 Trainable params: 1,658,113 (6.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

Model summary saved at models/tc/model_summary_tc.txt


#### Training the models

In [9]:
# Training the 1st Model "Robot Protective Stop" [Classification]
history_rb = utils.train_model(model_rb, trd_rb, trl_rb, vad_rb, val_rb, epochs=10, batch_size=32, checkpoint_filename="models/rb/best_model_rb.keras")
utils.save_model(model_rb, "models/rb/model_rb.keras")
utils.save_training_history(history_rb, "models/rb/training_history_rb.csv")

Epoch 1/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 81s 179ms/step - accuracy: 0.5093 - auc: 0.4679 - loss: 2.8686 - val_accuracy: 0.3626 - val_auc: 0.5489 - val_loss: 2.4324 - learning_rate: 0.0010
Epoch 2/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 50s 174ms/step - accuracy: 0.5387 - auc: 0.5167 - loss: 2.3047 - val_accuracy: 0.6022 - val_auc: 0.5050 - val_loss: 1.8568 - learning_rate: 0.0010
Epoch 3/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 46s 161ms/step - accuracy: 0.5394 - auc: 0.5019 - loss: 1.8498 - val_accuracy: 0.3415 - val_auc: 0.4873 - val_loss: 1.6417 - learning_rate: 0.0010
Epoch 4/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 58s 201ms/step - accuracy: 0.5262 - auc: 0.5031 - loss: 1.4850 - val_accuracy: 0.7293 - val_auc: 0.4918 - val_loss: 1.1204 - learning_rate: 0.0010
Epoch 5/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 48s 167ms/step - accuracy: 0.5547 - auc: 0.5522 - loss: 1.2214 - val_accuracy: 0.4300 - val_auc: 0.6002 - val_loss: 0.9711 - learning_rate: 0.0010
Epoch 6/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 50s 172ms/step - accuracy

In [10]:
# Training the 2nd Model "Grip Lost" [Classification]
history_gl = utils.train_model(model_gl, trd_gl, trl_gl, vad_gl, val_gl, epochs=10, batch_size=32, checkpoint_filename="models/gl/best_model_gl.keras")
utils.save_model(model_gl, "models/gl/model_gl.keras")
utils.save_training_history(history_gl, "models/gl/training_history_gl.csv")

Epoch 1/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 73s 171ms/step - accuracy: 0.6084 - auc: 0.5192 - loss: 2.7632 - val_accuracy: 0.8992 - val_auc: 0.4239 - val_loss: 2.2127 - learning_rate: 0.0010
Epoch 2/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 44s 152ms/step - accuracy: 0.5683 - auc: 0.5013 - loss: 2.2732 - val_accuracy: 0.6192 - val_auc: 0.3540 - val_loss: 1.8475 - learning_rate: 0.0010
Epoch 3/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 44s 153ms/step - accuracy: 0.5836 - auc: 0.4996 - loss: 1.8791 - val_accuracy: 0.8998 - val_auc: 0.3800 - val_loss: 1.4268 - learning_rate: 0.0010
Epoch 4/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 45s 155ms/step - accuracy: 0.5615 - auc: 0.5125 - loss: 1.5786 - val_accuracy: 0.9619 - val_auc: 0.6340 - val_loss: 1.1628 - learning_rate: 0.0010
Epoch 5/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 50s 173ms/step - accuracy: 0.5893 - auc: 0.5209 - loss: 1.3171 - val_accuracy: 0.9525 - val_auc: 0.5475 - val_loss: 1.0979 - learning_rate: 0.0010
Epoch 6/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 47s 162ms/step - accuracy

In [11]:
# Training the 3rd Model "Tool Current" [Regression]
history_tc = utils.train_model(model_tc, trd_tc, trl_tc, vad_tc, val_tc, epochs=10, batch_size=32, checkpoint_filename="models/tc/best_model_tc.keras")
utils.save_model(model_tc, "models/tc/model_tc.keras")
utils.save_training_history(history_tc, "models/tc/training_history_tc.csv")

Epoch 1/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 58s 194ms/step - loss: 2.3551 - mae: 0.6531 - rmse: 0.8385 - val_loss: 1.8168 - val_mae: 0.0769 - val_rmse: 0.1035 - learning_rate: 0.0010
Epoch 2/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 27s 180ms/step - loss: 1.7933 - mae: 0.3169 - rmse: 0.4032 - val_loss: 1.4306 - val_mae: 0.0665 - val_rmse: 0.0980 - learning_rate: 0.0010
Epoch 3/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 27s 179ms/step - loss: 1.3534 - mae: 0.1659 - rmse: 0.2150 - val_loss: 1.0647 - val_mae: 0.0454 - val_rmse: 0.0842 - learning_rate: 0.0010
Epoch 4/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 26s 172ms/step - loss: 0.9916 - mae: 0.1006 - rmse: 0.1349 - val_loss: 0.7720 - val_mae: 0.0547 - val_rmse: 0.0874 - learning_rate: 0.0010
Epoch 5/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 26s 176ms/step - loss: 0.7143 - mae: 0.0699 - rmse: 0.1042 - val_loss: 0.5537 - val_mae: 0.0436 - val_rmse: 0.0822 - learning_rate: 0.0010
Epoch 6/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 26s 177ms/step - loss: 0.5115 - mae: 0.0542 - rmse: 0.0881 - val_

In [12]:
# Freeing up memory
gc.collect()

1505